In [3]:
from exporter import ScholarExporter
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote
from datetime import date

/srv/conda/envs/notebook/lib/python3.7/site-packages/bs4/__init__.py:424: MarkupResemblesLocatorWarning: "https://scholar.google.com/citations?user=tlMCbgwAAAAJ" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning


AttributeError: 'NoneType' object has no attribute 'text'

In [40]:
main_url = "https://scholar.google.com/citations?user=tlMCbgwAAAAJ&sortby=pubdate"
re = requests.get(main_url)
parser = BeautifulSoup(re.text, "html.parser")                       
                       
scholar = parser.find('div', {'id': 'gsc_prf_in'}).text
papers = parser.body.find_all('tr', attrs={'class': 'gsc_a_tr'})

In [41]:
PAPER_TEMPLATE = """
<div class="card">
    <div class="card-publication">
        <div class="card-body card-body-left">
            <h4><a>{title}</a></h4>
            <p style="font-style: italic;">by {authors}</p>
            <p><b>{journal}</b></p>
        </div>
    </div>
    <div class="card-footer">
        <small class="text-muted">Published in <b>{year}</b> | 
        <a href="{citations_url}">Citations: <b>{n_citations}</b></a></small>
    </div>
</div>
"""

In [42]:
parsed_papers = []
for paper in papers:
            paper_soup = BeautifulSoup(str(paper), features="html.parser")
            try:
                citations_a = paper_soup.find('a', {'class': 'gsc_a_ac gs_ibl'})
                if citations_a is None:
                    citations_a = paper_soup.find('a', {'class': 'gsc_a_ac gs_ibl gsc_a_acm'})
                this_paper = {'title': paper_soup.find('a').text,
                              'year': paper_soup.find_all('span')[-1].text,
                              'n_citations': citations_a.text,
                              'citations_url': citations_a['href'],
                              'authors': paper_soup.find_all('div', {'class': 'gs_gray'})[0].text,
                              'journal': paper_soup.find_all('div', {'class': 'gs_gray'})[1].text}

                if not this_paper['n_citations']:
                    print("no citations")
                    this_paper['n_citations'] = "0"
                
                if this_paper['journal'].endswith(', ' + this_paper['year']):
                    this_paper['journal'] = this_paper['journal'][:-len(', ' + this_paper['year'])]
                parsed_papers.append(this_paper)
            except KeyError:
                print('Warning: error parsing paper (key error).')
            except IndexError:
                print('Warning: error parsing paper.')
            except AttributeError:
                print('Warning: error parsing paper.')

no citations
no citations
no citations
no citations
no citations


In [43]:
file = open("index.html","w")
for paper in parsed_papers:
              file.write(PAPER_TEMPLATE.format(**paper))
file.close()

In [44]:
s = ScholarExporter.from_user('tlMCbgwAAAAJ')  # Geoffrey Hinton user
t = s.export('index.html')

TypeError: 'NoneType' object is not subscriptable

KeyError: 'data-href'